Prepare meteorological data for WOFOST run on MCH gridded data

In [1]:
import pandas as pd
import xarray as xr
from pyproj import Transformer
import numpy as np
import datetime
from math import cos, sin, asin, pi, radians
import geopandas as gpd
import os

def get_MCH_data_lat_lon(startyear,endyear,variables,datadir,lat=None,lon=None,X=None,Y=None):

    """ Prepare Meteo Swiss data for WOFOST runs or run of phenology model
    
    Parameters
    ----------
        startyear: int
            startyear
        endyear: int
            endyear
        variables: list of strings
            list with variables to load
        lat: float
            latitude in degrees
        lon: float
            longiude in degrees
        X: X coordinate in LV95
        Y: Y coordinate in LV95
        

    Returns
    ----------            
       xarray.Dataset with variables at requested coordinate
    """
    if X and Y:
        print('Use LV95 coordinate system....')
    elif lat and lon:
        #make transformation
        print('transform lat lon to L95 coordinate system...')
        WGS84_to_LV95 = Transformer.from_crs("epsg:4326","epsg:2056",always_xy=True)
        X,Y=WGS84_to_LV95.transform(xx=lon,yy=lat)
    else:
        raise ValueError("really not enough coordinates specified.")
         
        
    startyear = int(startyear)
    endyear = int(endyear)

    time_series_list=[]
    
    for var in variables:
   
        #get list of datafiles #1961-2022
        datafiles=[datadir+str(var)+'D_ch01r.swiss.lv95_'+str(year)+'01010000_'+str(year)+'12310000.nc' for year in range(startyear,endyear+1)]
        #read datafiles
        print('reading data from {}...'.format(datadir))
        data=xr.open_mfdataset(datafiles,concat_dim = 'time',combine='nested',coords = 'minimal')
        
        #get data at lat lon
        time_series_list.append(data.sel(E=X,N=Y,method='nearest'))

    data_out=xr.merge(time_series_list)      

    return data_out

First, define directories of gridded exposure data (polygons), the MeteoSwiss data and the directory to store the output data

In [2]:
#produce MCH Data (comment if already done so)

#INPUT DATA
datadir_MCH='O:/Data-Raw/27_Natural_Resources-RE/99_Meteo_Public/MeteoSwiss_netCDF/__griddedData/lv95/'
datadir_exposure='C:/Users/F80840370/projects/scClim/climada/data/scClim/exposure/GIS/Weizen_Mais_Raps_Gerste_polygons.gpkg'
cwd = os.getcwd()

#OUTPUT DATA
OUTDIR='C:/Users/F80840370/projects/scClim/climada/data/scClim/exposure/phenology/TminTmax_grid/MeteoSwiss/'

startyear=1972
endyear=2022
variables=['TminD','TmaxD']
filename_base='TminTmax_daily_{}_{}'.format(startyear,endyear)

In [3]:
units=gpd.read_file(datadir_exposure)
units=units[units.n_fields>0]
units_lv95=units.to_crs(crs=2056)
units_lv95['X']=np.round(units_lv95.centroid.x)
units_lv95['Y']=np.round(units_lv95.centroid.y)

In [4]:
units_lv95['ID']=units_lv95.index


In [7]:
units_lv95.to_file("spatial_units.gpkg", driver="GPKG")

Read gridded MeteoSwiss Climate Data from Agroscope Server

In [41]:
#get list of datafiles #1961-2022
mch_data={}
for var in variables:
    datafiles=[datadir_MCH+str(var)+'_ch01r.swiss.lv95_'+str(year)+'01010000_'+str(year)+'12310000.nc' for year in range(startyear,endyear+1)]
    #read datafiles
    print('reading data for {} from {}...'.format(var,datadir_MCH))
    mch_data[var]=xr.open_mfdataset(datafiles,concat_dim = 'time',combine='nested',coords = 'minimal')


147475    2484500.0
147476    2484500.0
147477    2484500.0
148112    2485500.0
148116    2485500.0
            ...    
351650    2803500.0
351651    2803500.0
354204    2807500.0
363869    2822500.0
365790    2825500.0
Name: X, Length: 12742, dtype: float64

For each variable read MeteoSwiss data and create for each variable a DataFrame with time as column and spatial units (ID) as rows. Write one file per variable

In [30]:
for var in variables:
      df_out=pd.DataFrame({'time': mch_data.time.values})
      for unit in units_lv95:
      
            data=mch_data[var].sel(E=units_lv95.X,N=units_lv95.Y,method='nearest')
            df_out[unit.ID]=list(data.values)

      filename='{}_{}_{}'.format(var,startyear,endyear)
      df_out.to_csv(OUTDIR+filename+'.csv')


147475    147475
147476    147476
147477    147477
148112    148112
148116    148116
           ...  
351650    351650
351651    351651
354204    354204
363869    363869
365790    365790
Name: ID, Length: 12742, dtype: int64

,n_fields,area_ha,geometry,centroid
0,0.0,0.0,"POLYGON ((2254000.000 840000.001, 2254000.000 ...",POINT (2254500.000 840500.001)
1,0.0,0.0,"POLYGON ((2254000.000 841000.001, 2254000.000 ...",POINT (2254500.000 841500.001)
2,0.0,0.0,"POLYGON ((2254000.000 842000.001, 2254000.000 ...",POINT (2254500.000 842500.001)
3,0.0,0.0,"POLYGON ((2254000.000 843000.001, 2254000.000 ...",POINT (2254500.000 843500.001)
4,0.0,0.0,"POLYGON ((2254000.000 844000.001, 2254000.000 ...",POINT (2254500.000 844500.001)
...,...,...,...,...
454395,0.0,0.0,"POLYGON ((2963000.001 1475000.001, 2963000.001...",POINT (2963500.001 1475500.001)
454396,0.0,0.0,"POLYGON ((2963000.001 1476000.001, 2963000.001...",POINT (2963500.001 1476500.001)
454397,0.0,0.0,"POLYGON ((2963000.001 1477000.001, 2963000.001...",POINT (2963500.001 1477500.001)
454398,0.0,0.0,"POLYGON ((2963000.001 1478000.001, 2963000.001...",POINT (2963500.001 1478500.001)


Then create the daily time series for each PLZ in the phenological dataset and store it - takes some time

Voilá, you are ready to calibrate.